# 🌍 AI Agent for Climate Risk and Insurtech News
This notebook fetches news from reliable sources, categorizes them into insurance-related domains.

In [3]:
!pip install feedparser transformers pandas tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=adcc57bc425e16da8e3837781cf669d6a52ad93192eb4e3503a7ceddb1b455e1
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [4]:
import feedparser
import pandas as pd
from transformers import pipeline
from tqdm import tqdm


These are sample, reliable RSS feeds for insurance and climate news.


In [5]:
rss_feeds = {
    "Climate Risk": "https://news.un.org/feed/subscribe/en/news/topic/climate-change/feed/rss.xml",
    "InsureTech": "https://www.insurtechinsights.com/feed/",
    "Policies": "https://www.oecd.org/finance/insurance/rss.xml"
}


Fetch Articles from RSS Feeds

In [6]:
def fetch_articles():
    articles = []
    for domain, url in rss_feeds.items():
        feed = feedparser.parse(url)
        for entry in feed.entries[:5]:  # Limit to 5 for demo
            articles.append({
                "title": entry.title,
                "link": entry.link,
                "published": entry.published if 'published' in entry else '',
                "summary": entry.summary if 'summary' in entry else '',
                "domain": domain
            })
    return pd.DataFrame(articles)

df_articles = fetch_articles()
df_articles.head()


,title,link,published,summary,domain
0,Countries reach historic deal to cut shipping ...,https://news.un.org/feed/view/en/story/2025/04...,"Fri, 11 Apr 2025 12:00:00 +0000","After years of intense negotiations, countries...",Climate Risk
1,Myanmar earthquake latest: Entire communities ...,https://news.un.org/feed/view/en/story/2025/04...,"Tue, 01 Apr 2025 12:00:00 +0000",As the death toll continues to rise in Myanmar...,Climate Risk
2,Myanmar earthquake tragedy ‘compounds already ...,https://news.un.org/feed/view/en/story/2025/03...,"Mon, 31 Mar 2025 12:00:00 +0000",Entire communities in central Myanmar have bee...,Climate Risk
3,Myanmar earthquake: Search and rescue efforts ...,https://news.un.org/feed/view/en/story/2025/03...,"Sun, 30 Mar 2025 12:00:00 +0000",The human toll of the earthquake which devasta...,Climate Risk
4,"Myanmar quake: More than 1,600 reported killed...",https://news.un.org/feed/view/en/story/2025/03...,"Sat, 29 Mar 2025 12:00:00 +0000","Latest news reports indicate that over 1,600 h...",Climate Risk


In [7]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


In [8]:
def summarize_text(text):
    try:
        summary = summarizer(text[:1024], max_length=60, min_length=25, do_sample=False)[0]['summary_text']
        return summary
    except:
        return "Summary failed."

tqdm.pandas()
df_articles["summary_text"] = df_articles["summary"].progress_apply(summarize_text)



  0%|          | 0/10 [00:00<?, ?it/s]Your max_length is set to 60, but your input_length is only 38. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)

 20%|██        | 2/10 [00:09<00:39,  4.88s/it]Your max_length is set to 60, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)

 30%|███       | 3/10 [00:17<00:43,  6.25s/it]Your max_length is set to 60, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)

 40%|████      | 4/10 [00:28<00:46,  7.75s/it]Your max_length is set to 60, but your input_length is only 40. Since this i

Display and Export the Output

In [10]:
df_articles[["title", "summary_text", "domain", "link", "published"]].to_csv("climate_insurtech_news_summary.csv", index=False)
df_articles[["title", "summary_text", "domain", "link", "published"]].head()


,title,summary_text,domain,link,published
0,Countries reach historic deal to cut shipping ...,"After years of intense negotiations, countrie...",Climate Risk,https://news.un.org/feed/view/en/story/2025/04...,"Fri, 11 Apr 2025 12:00:00 +0000"
1,Myanmar earthquake latest: Entire communities ...,UN humanitarians have been rushing to support...,Climate Risk,https://news.un.org/feed/view/en/story/2025/04...,"Tue, 01 Apr 2025 12:00:00 +0000"
2,Myanmar earthquake tragedy ‘compounds already ...,Entire communities in central Myanmar have be...,Climate Risk,https://news.un.org/feed/view/en/story/2025/03...,"Mon, 31 Mar 2025 12:00:00 +0000"
3,Myanmar earthquake: Search and rescue efforts ...,The human toll of the earthquake continues to...,Climate Risk,https://news.un.org/feed/view/en/story/2025/03...,"Sun, 30 Mar 2025 12:00:00 +0000"
4,"Myanmar quake: More than 1,600 reported killed...","Over 1,600 killed in Myanmar with thousands i...",Climate Risk,https://news.un.org/feed/view/en/story/2025/03...,"Sat, 29 Mar 2025 12:00:00 +0000"
